In [99]:
%pip install transformers nltk scikit-learn numpy torch

Note: you may need to restart the kernel to use updated packages.


In [100]:
from IPython.display import display, Markdown, Latex
import tqdm as notebook_tqdm
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import movie_reviews
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from transformers import pipeline
import nltk
import pandas as pd
import numpy as np
import re

In [101]:
from sklearn.naive_bayes import (
    BernoulliNB,
    ComplementNB,
    MultinomialNB,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

Загрузим данные.

In [102]:
data = pd.read_csv("./IMDB Dataset.csv")
data.replace(to_replace={"sentiment": {"negative": 0, "positive": 1}}, inplace=True)
data["review"] = data["review"].apply(lambda r: r.replace("<br />", "\n"))

In [103]:
rm_quoted = re.compile(r"\".*?\"")

Сделаем версию без наименований в ковычках. Их исключение повышает точность моделей.

In [104]:
names = nltk.corpus.names.words()
cleanedup_data = data.copy(deep=True)
cleanedup_data["review"] = cleanedup_data["review"].apply(
    lambda r: re.sub(rm_quoted, "", r.replace("<br />", "\n"))
)

Также лемматизируем.

In [105]:
arrStopwords = stopwords.words("english")
arrStopwords.extend(["br", "<br>", "<br />"])
lemm_arr = []

wnl = WordNetLemmatizer()

for r in data.iterrows():
    rev = r[1]["review"]

    rev_no_quotes = re.sub(rm_quoted, "", rev)
    rev_tok = word_tokenize(rev_no_quotes)
    rev_tok = [w.lower() for w in rev_tok]

    rev_tok_filtered = [w for w in rev_tok if not w in arrStopwords]
    rev_tok_filtered = [w for w in rev_tok_filtered if w.isalpha()]

    rev_tok_lemmatized = np.array([wnl.lemmatize(w) for w in rev_tok_filtered])

    # reviews_tok_arr.append(rev_tok_filtered)
    lemm_arr.append((rev_tok_lemmatized, r[1]["sentiment"]))

lemm_data = pd.DataFrame(columns=["reviews", "sentiment"], data=lemm_arr)


In [106]:
processing_results = {"raw" : data, "cleaned" : cleanedup_data, "lemmatized" : lemm_data}

## Sentiment analyzer встроенный в NLTK

В NLTK есть предобученный определитель настроения называемый VADER. Можно решить задачу при помощи него.

In [107]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\danil\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [108]:
vader = SentimentIntensityAnalyzer()

In [109]:
vader_predictions = []

for r in cleanedup_data.iterrows():
    rev = r[1]["review"]

    vader_predictions.append(
        0 if vader.polarity_scores(rev)["compound"] < 0 else 1
    )

print(f"Точность: {accuracy_score(cleanedup_data['sentiment'], vader_predictions) * 100:.2f}%")

Точность: 69.76%


Уже так можно добится точности в районе 70%. На несколько процентов точность можно повысить, если добавить дополнительные признаки и использовать классификаторы из scikit-learn.

Сначала получим списоки отрицательных и положительных слов из корпуса NLTK.

In [110]:
stopword_en = nltk.corpus.stopwords.words("english")
stopword_en.extend([w.lower() for w in nltk.corpus.names.words()])

def skip_unwanted(pos_tuple):
    word, tag = pos_tuple
    if not word.isalpha() or word in stopword_en:
        return False
    if tag.startswith("NN"):
        return False
    return True

positive_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(nltk.corpus.movie_reviews.words(categories=["pos"]))
)]
negative_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(nltk.corpus.movie_reviews.words(categories=["neg"]))
)]

Потом выберем самые часто встречающиеся положительные и отрицательные леммы.

In [111]:
positive_fd = nltk.FreqDist(positive_words)
negative_fd = nltk.FreqDist(negative_words)

common_set = set(positive_fd).intersection(negative_fd)

for word in common_set:
    del positive_fd[word]
    del negative_fd[word]

top_100_positive_lem = {wnl.lemmatize(w) for w, count in positive_fd.most_common(100)}
top_100_negative_lem = {wnl.lemmatize(w) for w, count in negative_fd.most_common(100)}


Загрузим классифаеры из scikit-learn.

In [112]:
classifiers = {
    "BernoulliNB": BernoulliNB(),
    "ComplementNB": ComplementNB(),
    "MultinomialNB": MultinomialNB(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "MLPClassifier": MLPClassifier(max_iter=1000),
    "AdaBoostClassifier": AdaBoostClassifier(),
}

Функция для экстракции признаков. Она собирает количество негативных и позитивных слов, а также значения выдаваемые VADER.

In [113]:
def extract_features(text):
    features = dict()
    pos_word_count = 0
    neg_word_count = 0

    for r in wnl.lemmatize(text):
        for w in r:
            if w in top_100_negative_lem:
                neg_word_count += 1
            elif w in top_100_positive_lem:
                pos_word_count += 1

    vader_scores = vader.polarity_scores(text)
    
    features["compound"] = vader_scores["compound"] + 1
    features["neg"] = vader_scores["neg"]
    features["pos"] = vader_scores["pos"]
    features["pos_word_count"] = pos_word_count
    features["neg_word_count"] = neg_word_count

    return features

Собирём признаки для всего датасета

In [114]:
featurelist = []
feature_names = list(extract_features("I like pickles"))
data_column_names = feature_names + ["sentiment"]

for r in cleanedup_data.iterrows():
    r_features_dict = extract_features(r[1]["review"])
    r_features_dict["sentiment"] = r[1]["sentiment"]
    featurelist.append(r_features_dict)

features_df = pd.DataFrame(data=featurelist)

Используем признаки как вводный массив для разных моделей sklearn. Как видим точность некоторых моделей привосходит точность VADER самого по себе. Теоретически можно попробовать вывести ещё больше значимых признаков из текстов данного датасета и увеличить тем самым точность ещё выше

In [115]:
train_data, test_data = train_test_split(features_df, train_size=0.85)

for name, sklearn_classifier in classifiers.items():
    cls = sklearn_classifier

    x_train = train_data.drop(columns=["sentiment"])
    y_train = train_data["sentiment"]

    x_test = test_data.drop(columns=["sentiment"])
    y_test = test_data["sentiment"]

    model = cls.fit(X=x_train, y=y_train)

    predictions = model.predict(x_test)

    accuracy = accuracy_score(y_test, predictions)
    print(F"{accuracy:.2%} - {name}")

53.15% - BernoulliNB
70.08% - ComplementNB
69.99% - MultinomialNB
69.39% - KNeighborsClassifier
64.37% - DecisionTreeClassifier
70.64% - RandomForestClassifier
72.76% - MLPClassifier
73.07% - AdaBoostClassifier


Главным приемуществом данного метода является его скорость. Обработать весь датасет при помощи VADER можно за секунды

In [116]:
%%timeit -n1 -r1 -o
dummy_predictions = []
for r in cleanedup_data.iterrows():
    rev = r[1]["review"]

    dummy_predictions.append(
        0 if vader.polarity_scores(rev)["compound"] < 0 else 1
    )

1min 30s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 1min 30s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [117]:
vader_worktime = _

In [118]:
display(Markdown(f"### **Главным приемуществом данного метода является его скорость. Обработать весь датасет при помощи VADER можно за {vader_worktime.average:.2f} секунды**"))

### **Главным приемуществом данного метода является его скорость. Обработать весь датасет при помощи VADER можно за 91.00 секунды**

## Модели с Hugging Face

Также для анализа можно использовать модели предтренированные работоприятные с Hugging Face. Например [DistilBERT base uncased finetuned SST-2](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english). Главная проблема заключается в том, что глубинные нейронные сети почти в 100 раз медленней алгоритма VADER и на анализ всего датасета уйдет более 7 часов. Так что для демонстрации и предварительного вычисления точности мы используем небольшое подмножество датасета из 1000 отзывов. Данная модель также не может обрабатывать более 512 токенов, так что нам придётся обрезать каждый отзыв.

In [119]:
sentiment_pipeline = pipeline(task="sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

In [120]:
model_processing_time = %timeit sentiment_pipeline(data["review"][randint(1, 50000)][:500])

114 ms ± 5.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [121]:
vader_processing_time = %timeit vader.polarity_scores(data["review"][randint(1, 50000)])

1.64 ms ± 83.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [122]:
ml_test_dataset = shuffle(data, random_state=42).iloc[:1000].copy(deep=True)
ml_test_dataset["review"] = ml_test_dataset["review"].apply(lambda r: r[:500])

In [123]:
predictions = np.array([
    0 if p["label"] == "NEGATIVE" else 1
    for p in sentiment_pipeline(list(ml_test_dataset["review"]))
])

In [124]:
ml_accuracy = accuracy_score(ml_test_dataset["sentiment"], predictions)
print(f"{ml_accuracy * 100:.2f}% - DistilBERT")

80.70% - DistilBERT


Тем не менее DistilBERT дает точность предсказаний гораздо выше других моделей - в районе 80%